In [22]:
import os
import math

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
import h5py
import torch
from torchviz import make_dot

import wf_func as wff

In [23]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=160., scale=40.)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [24]:
Thres = {'xiaopeip':0.2, 'lucyddm':0.2, 'fftrans':0.1, 'findpeak':0.1, 'threshold':0.1, 'omp':0}
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
window = 1029
p = spe_pre[0]['parameters']
t = np.arange(window).astype(np.float)
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
plt.close()

In [25]:
fig = plt.figure(figsize=(8, 6))
t = np.arange(-4 * 5, 5 * 20, 0.1)
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(t, wff.convolve_exp_norm(t, 20, 0), label=r'$(20,0)$', color='g')
ax.plot(t, wff.convolve_exp_norm(t, 0, 5), label=r'$(0,5)$', color='r')
ax.plot(t, wff.convolve_exp_norm(t, 20, 5), label=r'$(20,5)$', color='b')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(5))
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.legend(title=r'$(\tau, \sigma)/\si{ns}$', loc='upper right')
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
# ax.annotate(r'$t_{0}$', xy=(0, 0), xytext=(5, 0.01), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()

In [26]:
Mu = 4
Tau = 20
Sigma = 5
file = '4.0-20-5'

In [27]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]

In [52]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.9, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(500) / gmu, chargehist(np.arange(500)), label='$\mathrm{ChargePDF}$')
ax.set_xlabel(r'$\mathrm{Charge}$')
ax.set_xlim(0, 400 / gmu)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.grid()
ax.legend()
fig.savefig('Note/figures/chargehist.pgf')
fig.savefig('Note/figures/chargehist.pdf')
fig.clf()
plt.close(fig)

In [29]:
xn = np.linspace(-5, 5, 600)
yn = np.linspace(-5, 5, 600)
xv, yv = np.meshgrid(xn, yn)
pos = np.dstack((xv, yv))

m = np.array([[-1., 1.5], [-1., 1.]])
s = np.array([[1., .5], [.6, .8]])
a = np.array([[.4, .6], [.4, .6]])
rho = np.array([[0.5, -0.7], [0.4, 0.4]])

ymax = max(np.max(normcombine(xn, m[0, :], s[0, :], a[0, :])), np.max(normcombine(yn, m[1, :], s[1, :], a[1, :])))
fig = plt.figure(figsize=(5, 5))
ax0 = fig.add_axes((.1, .85, .75, .12))
ax0.axis('off')
ax0.plot(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), color='b')
ax0.fill_between(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), facecolor='blue', alpha=0.5, color=None)
ax0.set_xticks([])
ax0.set_yticks([])
ax0.set_ylim(0, ymax)
ax1 = fig.add_axes((.85, .1, .12, .75))
rot = transforms.Affine2D().rotate_deg(-90)
base = plt.gca().transData
ax1.axis('off')
ax1.plot(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), color='b', transform=rot+base)
ax1.fill_between(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), facecolor='blue', alpha=0.5, color=None, transform=rot+base)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlim(0, ymax)
ax2 = fig.add_axes((.1, .1, .75, .75))
ax2.contourf(xv, yv, normcombine2d(pos, m, s, a, rho), cmap=cm.PuBu)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(r'$\mu$')
ax2.set_ylabel(r'$\nu$')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
fig.savefig('Note/figures/wdistdemo.pgf')
fig.savefig('Note/figures/wdistdemo.pdf')
fig.clf()
plt.close(fig)

In [30]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure(figsize=(15, 4))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 3, figure=fig, left=0.07, right=0.97, top=0.97, bottom=0.2, wspace=0.3, hspace=0.3)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])
ax0.bar(t, a, color='r')
ax0.set_xlabel(r'$a$')
ax0.set_ylabel(r'$\mathrm{weight}\ a$')
ax0.set_xticks(t)
ax0.set_ylim(0, ax0.get_ylim()[1] * 1.05)
ax1.bar(t, b1, color='b')
ax1.set_xlabel(r'$b_{1}$')
ax1.set_ylabel(r'$\mathrm{weight}\ b_{1}$')
ax1.set_xticks(t)
ax1.set_ylim(0, ax1.get_ylim()[1] * 1.05)
ax2.bar(t, b2, color='b')
ax2.set_xlabel(r'$b_{2}$')
ax2.set_ylabel(r'$\mathrm{weight}\ b_{2}$')
ax2.set_xticks(t)
ax2.set_ylim(0, ax2.get_ylim()[1] * 1.05)
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [31]:
i = 2
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

,TriggerNo,PMTId,HitTime,Charge
0,2,0,501.57,186.74
1,2,0,509.76,236.77
2,2,0,519.14,192.78
3,2,0,523.66,150.81
4,2,0,540.19,199.18


In [32]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 20 - 1) * 20
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 20 + 1) * 20, xmin + 200)

In [33]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'].min(), 0, wave.sum() / gmu, color='r', label='Record')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
rise = truth['HitPosInWindow'].min()
vsum = wave.sum()
ax2.annotate(fr'${rise:.{4}}\mathrm{{ns}}, {vsum:.{4}}\mathrm{{mV}}\cdot\mathrm{{ns}}$', xy=(truth['HitPosInWindow'].min(), wave.sum()*2/3), xytext=(truth['HitPosInWindow'].min()+120, wave.sum()*1/2), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax2.set_ylim(-5 / gmu, ax2.get_ylim()[1] * 1.05)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/previous.pgf')
fig.savefig('Note/figures/previous.pdf')
fig.clf()
plt.close(fig)

In [34]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(wave, label='Waveform')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax.legend()
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.clf()
plt.close(fig)

fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Charge}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(bottom=0)
ax.legend()
fig.savefig('Note/figures/charge.pgf')
fig.savefig('Note/figures/charge.pdf')
fig.clf()
plt.close(fig)

In [35]:
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'] / gmu, color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5 / gmu)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [36]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise
truth['Charge']

926.0
966.2815566497009


array([186.73723658, 236.77207449, 192.77997668, 150.80900479,
       199.1832641 ])

In [57]:
t = np.load('result/takara/char/Channel00/testing_record_2021-03-31_04:38:38.npz')['arr_0']
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(1, len(t)+1), t, label='W-dist', color='C1')
ax.set_xlabel(r'$\mathrm{epoch}$')
ax.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.savefig('Note/figures/epoch.pdf')
fig.clf()
plt.close(fig)

In [37]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['threshold'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [505 506 507 508 509 510 511 512 513 514 515 516 517 518 519 520 521 522
 523 524 525 526 527 528 538 539 540 541 542], Weight = [0.16266642 0.16266642 0.22259615 0.23115754 0.23115754 0.23971893
 0.21403476 0.1797892  0.17122781 0.15410503 0.18835059 0.21403476
 0.19691198 0.20547337 0.23115754 0.25684172 0.26540311 0.26540311
 0.23971893 0.23115754 0.21403476 0.1797892  0.16266642 0.15410503
 0.15410503 0.15410503 0.18835059 0.17122781 0.14554364]
wdist = 3.523124673822959, cdiff = -40.29957206987044
RSS = 2283.473341058975
7.88953884361257


In [38]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['findpeak'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
for i in range(len(loc)):
    ax.annotate('', xy=(loc[i], amp[i]+5), xytext=(loc[i], amp[i]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [510 523 541], Weight = [2.13223684 2.13223684 1.52302632]
wdist = 4.587636716073468, cdiff = -40.29957206987073
RSS = 2518.4630714961395
8.697379065494943


In [39]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['fftrans'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.yaxis.get_major_formatter().set_powerlimits((0, 1))
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [498 499 500 501 502 503 504 505 506 507 508 509 510 511 512 518 519 520
 521 522 523 524 525 538 539 540 541 542 543], Weight = [0.1314074  0.16216409 0.17104016 0.16014143 0.13831147 0.11807287
 0.1111744  0.12427555 0.15630264 0.19847007 0.23705208 0.25802747
 0.25203999 0.21796616 0.16382248 0.16241357 0.22946178 0.2815106
 0.30597632 0.29820744 0.26202181 0.20759059 0.1475815  0.17114891
 0.22728722 0.25886343 0.25700641 0.22070102 0.15746117]
wdist = 1.7245390537829974, cdiff = -40.29957206987073
RSS = 239.4998434939463
2.434222247878097


In [40]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, Thres['lucyddm'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [500 502 509 510 518 519 523 524 540], Weight = [0.24892833 0.92563176 0.61379691 0.86404593 0.27611932 0.74232786
 0.70408306 0.32445137 1.08811545]
wdist = 1.007986888580096, cdiff = -40.29957206987044
RSS = 68.53685960757073
2.756298909152463


In [46]:
with h5py.File('result/takara/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/takara.pgf')
fig.savefig('Note/figures/takara.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [501 509 510 519 523 540], Weight = [1.0563685  0.84381914 0.61144865 1.0334349  1.0406994  1.2017294 ]
wdist = 0.6385719330051693, cdiff = -40.299587328362875
RSS = 62.77188472405516
2.8618459911495506


In [55]:
with h5py.File('result/mcmc/char/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    photoelec = ipt['photoelectron'][:]
s = photoelec[(photoelec['TriggerNo'] == eid) & (photoelec['ChannelID'] == cid)]
pet = s['HitPosInWindow']
pwe = s['Charge']
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/mcmc.pgf')
fig.savefig('Note/figures/mcmc.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [500. 502. 509. 510. 518. 519. 523. 524. 540.], Weight = [0.24892833 0.92563176 0.61379691 0.86404593 0.27611932 0.74232786
 0.70408306 0.32445137 1.08811545]
wdist = 1.0079868885800962, cdiff = -40.299572069870585
RSS = 68.53685960757088
41.662806111922066


In [47]:
pet, pwe = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['xiaopeip'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
pwe = pwe / gmu
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [501 502 509 510 518 519 523 524 540 541], Weight = [0.69077108 0.3557919  0.72159512 0.73461505 0.20339688 0.82792813
 0.73676413 0.30379321 1.00654504 0.20629946]
wdist = 0.7102135123860166, cdiff = -40.299572069870585
RSS = 44.970817986273985
3.1437142805343683


In [48]:
p = spe_pre[cid]['parameters']
nsp = 4
nstd = 3
D = 100
factor = np.linalg.norm(spe_pre[cid]['spe'])
n = max(math.ceil(Mu / math.sqrt(Tau ** 2 + Sigma ** 2)), 1)
A, wave_r, tlist, t0_init, t0_init_delta, char_init = wff.initial_params(wave, spe_pre[cid], Mu, Tau, Sigma, gmu,0.2, p, nsp, nstd, is_t0=True, is_delta=False, n=n, nshannon=1)
A = A / factor
xmmse, xmmse_star, psy_star, nu_star, T_star, d_tot_i, d_max_i = wff.fbmpr_fxn_reduced(wave_r, A, min(-1e-3+1, Mu / len(tlist)), spe_pre[cid]['std'] ** 2, (gsigma * factor / gmu) ** 2, factor, D, stop=0)
xmmse_most = xmmse_star[0]
pet = tlist[xmmse_most > 0]
cha = xmmse_most[xmmse_most > 0] / factor
pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe
fig = plt.figure(figsize=(8, 6))
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fbmp.pgf')
fig.savefig('Note/figures/fbmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe * gmu, gmu=gmu, Tau=Tau, Sigma=Sigma, mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 5
truth HitPosInWindow = [501.56845877 509.75870458 519.14392113 523.65666631 540.18753678], Weight = [1.16713044 1.47985426 1.2048983  0.94257462 1.24491962]
truth RSS = 1107.7123357126038
HitPosInWindow = [501. 509. 510. 519. 524. 540.], Weight = [1.09651972 0.85809184 0.69799671 1.23789162 0.94111212 1.25277585]
wdist = 0.47725082381076533, cdiff = 7.201560189359567
RSS = 24.30864373545687
2.9104827461848686


In [ ]:
with h5py.File('result/mcmc/solu/' + file + '.h5', 'r', libver='latest', swmr=True) as soluf, h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as wavef:
    start = wavef['SimTruth/T'][:]
    time = soluf['starttime'][:]
data = time['tswave'] - start['T0']
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes((.1, .45, .85, .45))
ax.hist(data, bins=100, density=1)
ax.set_ylabel('Normalized Count')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(data.min() - 0.05, data.max() + 0.05)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
axb = fig.add_axes((.1, .15, .85, .3))
axb.boxplot(data, vert=False, sym='', patch_artist=True)
axb.set_xlabel(r'$\Delta t_{0}/\si{ns}$')
axb.set_yticks([])
axb.set_xlim(ax.get_xlim())
fig.savefig('Note/figures/mcmct0hist.pgf')
fig.savefig('Note/figures/mcmct0hist.pdf')
plt.close(fig)

In [54]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 5)
        wdist_stats[i, 5] = np.percentile(dtwpi, 95)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 5)
        edist_stats[i, 5] = np.percentile(dtepi, 95)

    L = len(dt)
    data = dt['wdist']
    vali = np.abs(data - np.mean(data)) < 7 * np.std(data, ddof=-1)
    data = data[vali]
    fig = plt.figure(figsize=(8, 6))
    ax1 = fig.add_axes((.1, .12, .55, .8))
#     ey = np.vstack([wdist_stats[:, 0]-wdist_stats[:, 4], wdist_stats[:, 5]-wdist_stats[:, 0]])
#     ax1.errorbar(np.arange(1, l+1), wdist_stats[:, 0], yerr=ey, label=r'$W-dist^{95\%}_{5\%}$')
    ax1.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax1.plot(np.arange(1, l + 1), wdist_stats[:, 0], label='W-dist')
    ax1.set_xlim(0, l + 1)
    ax1.set_ylim(data.min() - 0.05, np.percentile(data, 98) + 0.05)
    ax1.set_xlabel(r'$N_{pe}$')
    ax1.set_ylabel(r'$\mathrm{W-dist}/\si{ns}$')
    # ax.set_title('W-dist vs NPE stats')
    ax1.legend()
    ax2 = fig.add_axes((.65, .12, .33, .8))
    ax2.hist(data, bins=np.arange(0, data.max(), np.percentile(data, 98) / 40), density=1, orientation='horizontal')
    ax2.set_xlabel('Normalized Count')
    ax2.set_xlim(0, ax2.get_xlim()[1] * 1.05)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_ylim(ax1.get_ylim())
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)